## 1. Загрузка данных

In [29]:
#importing libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [30]:
df = pd.read_csv('/Users/DanilBee/Desktop/Yandex_projects/mlInsurance/insurance.csv')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Данные загружены , и проблем с ними пока не обнаружено

## 2. Умножение матриц

In [33]:
#splitting into training and testing data
df_train, df_test = train_test_split(df, test_size = 0.4, random_state = 42)
target_test = df_test['Страховые выплаты']
target_train = df_train['Страховые выплаты']
features_test = df_test.drop('Страховые выплаты', axis = 1)
features_train = df_train.drop('Страховые выплаты', axis = 1)

In [34]:
#making the features from df to matrixes
vector_test = features_test.values
vector_train = features_train.values

In [35]:
vector_test

array([[1.00e+00, 2.80e+01, 5.61e+04, 0.00e+00],
       [1.00e+00, 3.20e+01, 4.19e+04, 2.00e+00],
       [1.00e+00, 3.00e+01, 2.63e+04, 0.00e+00],
       ...,
       [1.00e+00, 5.50e+01, 3.31e+04, 1.00e+00],
       [0.00e+00, 1.90e+01, 4.23e+04, 1.00e+00],
       [1.00e+00, 5.90e+01, 5.17e+04, 2.00e+00]])

In [49]:
#generating random squared matrix with the width the same as in matrixes
rand_matrix = np.random.rand(4,4)
#checking the matrix for bein invertible
ver_matrix = np.linalg.inv(rand_matrix)

In [54]:
featuresv_test.shape[0]

2000

In [37]:
#scalar multiplication
featuresv_test = vector_test @ ver_matrix
featuresv_train = vector_train @ ver_matrix

In [38]:
#training on just regular data
model = LinearRegression()
model.fit(features_train, target_train)
prediction = model.predict(features_test)
r2_score(target_test, prediction)

0.42680182088269225

In [39]:
#training on crypted data
modelv = LinearRegression()
modelv.fit(featuresv_train, target_train)
predictionv = modelv.predict(featuresv_test)
r2_score(target_test, predictionv)

0.4268018208829526

**Ответ:** Умножили признаки на квадратную,обратимую матрицу и качество линейной регрессии не изменилось

**Обоснование:** 
Докажем,что :
$$ a = Xw = X'w' =a'      (1)$$

Также дано ,что:
$$ w = (X^T X)^{-1} X^T y      (2)$$

Пусть $M$ обратимая, квадратная матрица ,размер которой равен ширине Х
<br>
Тогда:
$$ X' = XM (3)$$
<br>
$$ w' = argminMSE(XMw,y) (4)$$
<br>
<br>
Решение:
<br>
В формулу (2) вставим (3) и получим :
$$ w' = ((XM)^T XM)^{-1} (XM)^T y $$
<br>
По свойствам обратных матриц , раскрываем скобки:
$$ w' = M^{-1}((XM)^T X)^{-1}(XM)^T y $$
<br>
Далее также приминяем свойство транспонированных матриц, раскрываем скобки:
$$ w' = M^{-1}(M^T X^T X)^{-1} M^T X^T y $$
<br>
Далее по свойству обратных матриц также вытаскиваем из скобки $(M^T..)^{-1}$:
$$ w' = M^{-1}(X^T X)^{-1}(M^T)^{-1} M^T X^T y $$
<br>
Далее можем сократить выражение $(M^{T})^{-1} M^T$:
$$ w' = M^{-1}(X^T X)^{-1} X^T y $$
<br>
Заметили ,что данная часть выражения $ (X^T X)^{-1} X^T y $ равняется $ w $:
$$ w' = M^{-1}w (5)$$
<br>
Теперь подставим это в формулу для нахождения $ a' = X'w'$ и получим:
$$ a' = X'M^{-1}w $$
Также подставим из (3) и получим:
$$ a' = XMM^{-1}w $$
$ MM^{-1} $ сокращается
В итоге имеем:
$$ a' = Xw $$
А значит:
$$ a' = a $$
<br>
Таким образом доказали ,что при умножении матрицы $X$ на обратимую,квадратную матрицу $M$ качество линейной регрессии не изменится

## 3. Алгоритм преобразования

In [58]:
class encryptedModel:
    def fit(self, train_features, train_target):
        self.R = np.linalg.inv(np.random.rand(4,4))
        self.M = train_features @ self.R
        X = np.concatenate((np.ones((self.M.shape[0], 1)), self.M), axis=1)
        y = train_target
        w = ((np.linalg.inv(X.T.dot(X))) @ X.T) @ y
        self.w = w[1:]
        self.w0 = w[0]
    def predict(self, test_features):
        return (test_features @ self.R) @ self.w + self.w0

In [60]:
model = encryptedModel()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('R2 score', r2_score(target_test, predictions))

R2 score 0.42680182416579604


**Обоснование**

Так как в пункте 2 мы доказали ,что при умножении на обратимую,квадратную матрицу качество линейной регрессии не ухудшится ,то запрограммировали модель линейной регрессии так ,чтобы она создавала рандомную матрицу , ее инвертировало для проверки на ошибку. Далее также с помощью формул высчитывает весы и предсказание с поправкой ,что данные зашифрованы 

## 4. Проверка алгоритма

In [61]:
modelSklearn = LinearRegression()
modelCustom = encryptedModel()
modelSklearn.fit(features_train, target_train)
modelCustom.fit(features_train, target_train)
predictionSklearn = modelSklearn.predict(features_test)
predictionCustom = modelCustom.predict(features_test)
print('R2 score on Sklearn model:', r2_score(target_test, predictionSklearn))
print('R2 score on Custom model with encryption:', r2_score(target_test, predictionCustom))

R2 score on Sklearn model: 0.42680182088269225
R2 score on Custom model with encryption: 0.4268018710115804


Модели показали одинаковые результаты как с проебразованием , так и без преобразования